In [1]:
import os
import numpy as np
import shutil

## Generate Jobs

In [2]:
raw_video_clip_path = r"/project/6055479/lingheng/cyc_safety_project/vic_cyc_legacy_pass_event_clip/"
face_blurred_video_clip_path = r"/project/6055479/lingheng/cyc_safety_project/cyc_face_and_plate_blur/"

In [3]:
job_script_root_dir = '/project/6055479/lingheng/cyc_safety_job_scripts_face_blur'  # Where to save job scripts
job_output_root_dir = '/project/6055479/lingheng/cyc_safety_job_output_face_blur'   # Where to save job outputs
if not os.path.exists(job_script_root_dir):
    os.makedirs(job_script_root_dir)
else:
    # If exist, remove to avoid recreate job scripts
    shutil.rmtree(job_script_root_dir)
    os.makedirs(job_script_root_dir)
    
if not os.path.exists(job_output_root_dir):
    os.makedirs(job_output_root_dir)

In [29]:
raw_video_clip_list = os.listdir(raw_video_clip_path)
raw_video_clip_list.sort()
raw_video_clip_list

['PassingEvent_Duration10s_000001.mp4',
 'PassingEvent_Duration10s_000003.mp4',
 'PassingEvent_Duration10s_000005.mp4',
 'PassingEvent_Duration10s_000006.mp4',
 'PassingEvent_Duration10s_000007.mp4',
 'PassingEvent_Duration10s_000008.mp4',
 'PassingEvent_Duration10s_000009.mp4',
 'PassingEvent_Duration10s_000010.mp4',
 'PassingEvent_Duration10s_000011.mp4',
 'PassingEvent_Duration10s_000012.mp4',
 'PassingEvent_Duration10s_000015.mp4',
 'PassingEvent_Duration10s_000016.mp4',
 'PassingEvent_Duration10s_000017.mp4',
 'PassingEvent_Duration10s_000018.mp4',
 'PassingEvent_Duration10s_000019.mp4',
 'PassingEvent_Duration10s_000020.mp4',
 'PassingEvent_Duration10s_000021.mp4',
 'PassingEvent_Duration10s_000022.mp4',
 'PassingEvent_Duration10s_000023.mp4',
 'PassingEvent_Duration10s_000024.mp4',
 'PassingEvent_Duration10s_000025.mp4',
 'PassingEvent_Duration10s_000026.mp4',
 'PassingEvent_Duration10s_000027.mp4',
 'PassingEvent_Duration10s_000028.mp4',
 'PassingEvent_Duration10s_000029.mp4',


In [30]:
len(raw_video_clip_list)

18527

In [13]:
# no_done_list = raw_video_clip_list
no_done_list = no_done_list

batch_size = 25 # 100                                             # Specify how many clips will be precessed in one job
start_idx = np.arange(0, len(no_done_list), batch_size)   # Specify the start clip id included within one job
end_idx = np.append(start_idx[1:], [len(no_done_list)])   # Specify the end clip id included within one job

for b_i in range(len(start_idx)):
    # Get start and end id for the batch
    process_start_id = start_idx[b_i]
    process_end_id = end_idx[b_i]
    print('batch {}, start={}, end={}'.format(b_i, process_start_id, process_end_id))
    #
    job_script_name = 'job_face_blur-{}-{}-{}.sh'.format(b_i, process_start_id, process_end_id)
    job_output_name = 'output_face_blur-{}-{}-{}_R-%x.%j.out'.format(b_i, process_start_id, process_end_id)
    
    # Write batch job script
    with open(os.path.join(job_script_root_dir, job_script_name), 'w', encoding='utf8', newline='\n') as f:
        f.write('''\
#!/bin/bash
#SBATCH --account=def-rgorbet
#SBATCH --nodes=1                # When requesting GPUs per node the number of nodes must be specified.
#SBATCH --ntasks=5               # number of MPI processes
#SBATCH --mem-per-cpu=8000M          # memory; default unit is megabytes
#SBATCH --time=0-10:0         # time (DD-HH:MM)
#SBATCH --output={0}/{1}

module load StdEnv/2020  gcc/9.3.0  cuda/11.4 opencv/4.7.0 python/3.8 
source /home/lingheng/cyc_env/bin/activate
cd /project/6055479/lingheng/DashcamCleaner/
pip install -r requirements.txt
pip uninstall -y opencv-python
pip install opencv-python

                '''.format(job_output_root_dir, job_output_name))
    # Add a command for each video clip within the batch 
    for video_id in range(process_start_id, process_end_id):
        no_done_list[video_id]
        with open(os.path.join(job_script_root_dir, job_script_name), 'a', encoding='utf8', newline='\n') as f:
            f.write('''
python /project/6055479/lingheng/DashcamCleaner/dashcamcleaner/cli.py -i {}{} -o {} -w 720p_medium_v8.pt --blur_size 25 --quality 7
                    '''.format(raw_video_clip_path, 
                               no_done_list[video_id],
                               face_blurred_video_clip_path))

batch 0, start=0, end=25
batch 1, start=25, end=50
batch 2, start=50, end=75
batch 3, start=75, end=100
batch 4, start=100, end=125
batch 5, start=125, end=150
batch 6, start=150, end=175
batch 7, start=175, end=200
batch 8, start=200, end=225
batch 9, start=225, end=250
batch 10, start=250, end=275
batch 11, start=275, end=300
batch 12, start=300, end=325
batch 13, start=325, end=350
batch 14, start=350, end=375
batch 15, start=375, end=400
batch 16, start=400, end=425
batch 17, start=425, end=450
batch 18, start=450, end=475
batch 19, start=475, end=500
batch 20, start=500, end=525
batch 21, start=525, end=550
batch 22, start=550, end=575
batch 23, start=575, end=600
batch 24, start=600, end=625
batch 25, start=625, end=650
batch 26, start=650, end=675
batch 27, start=675, end=700
batch 28, start=700, end=725
batch 29, start=725, end=750
batch 30, start=750, end=775
batch 31, start=775, end=800
batch 32, start=800, end=825
batch 33, start=825, end=850
batch 34, start=850, end=875
bat

## Submit Jobs

In [14]:
job_script_list = os.listdir(job_script_root_dir)
job_script_list.sort()
job_script_list

['job_face_blur-0-0-25.sh',
 'job_face_blur-1-25-50.sh',
 'job_face_blur-10-250-275.sh',
 'job_face_blur-11-275-300.sh',
 'job_face_blur-12-300-325.sh',
 'job_face_blur-13-325-350.sh',
 'job_face_blur-14-350-375.sh',
 'job_face_blur-15-375-400.sh',
 'job_face_blur-16-400-425.sh',
 'job_face_blur-17-425-450.sh',
 'job_face_blur-18-450-475.sh',
 'job_face_blur-19-475-500.sh',
 'job_face_blur-2-50-75.sh',
 'job_face_blur-20-500-525.sh',
 'job_face_blur-21-525-550.sh',
 'job_face_blur-22-550-575.sh',
 'job_face_blur-23-575-600.sh',
 'job_face_blur-24-600-625.sh',
 'job_face_blur-25-625-650.sh',
 'job_face_blur-26-650-675.sh',
 'job_face_blur-27-675-700.sh',
 'job_face_blur-28-700-725.sh',
 'job_face_blur-29-725-750.sh',
 'job_face_blur-3-75-100.sh',
 'job_face_blur-30-750-775.sh',
 'job_face_blur-31-775-800.sh',
 'job_face_blur-32-800-825.sh',
 'job_face_blur-33-825-850.sh',
 'job_face_blur-34-850-875.sh',
 'job_face_blur-35-875-900.sh',
 'job_face_blur-36-900-922.sh',
 'job_face_blur-4-10

In [15]:
len(job_script_list)

37

In [16]:
for job_i, job_script in enumerate(job_script_list):
    job_script_path = os.path.join(job_script_root_dir, job_script)
    print('Submitting {}: {}'.format(job_i, job_script_path))
    !sbatch {job_script_path} 
    print("#########################################################################################")

Submitting 0: /project/6055479/lingheng/cyc_safety_job_scripts_face_blur/job_face_blur-0-0-25.sh
Submitted batch job 5794772
#########################################################################################
Submitting 1: /project/6055479/lingheng/cyc_safety_job_scripts_face_blur/job_face_blur-1-25-50.sh
Submitted batch job 5794773
#########################################################################################
Submitting 2: /project/6055479/lingheng/cyc_safety_job_scripts_face_blur/job_face_blur-10-250-275.sh
Submitted batch job 5794774
#########################################################################################
Submitting 3: /project/6055479/lingheng/cyc_safety_job_scripts_face_blur/job_face_blur-11-275-300.sh
Submitted batch job 5794775
#########################################################################################
Submitting 4: /project/6055479/lingheng/cyc_safety_job_scripts_face_blur/job_face_blur-12-300-325.sh
Submitted batch job 5794776
##

## Check Results

In [4]:
blurred_video_list = os.listdir(face_blurred_video_clip_path)

In [5]:
len(blurred_video_list)

18527

In [6]:
raw_video_list = os.listdir(raw_video_clip_path)

In [7]:
len(raw_video_list)

18527

### Remove unfinished video

In [8]:

unfinished_list = []
for tmp_video in blurred_video_list:
    if '_copy' in tmp_video:
        print(tmp_video)
        os.remove(os.path.join(face_blurred_video_clip_path, tmp_video))
        unfinished_list.append(tmp_video)

In [9]:
len(unfinished_list)

0

### Get the list of videos that are not blurred
rerun job generate and submit to process the unblurred videos

In [10]:
blurred_video_list = os.listdir(face_blurred_video_clip_path)
no_done_list = []
for tmp_video in raw_video_list:
    if tmp_video not in blurred_video_list:
        print(tmp_video)
        no_done_list.append(tmp_video)

In [11]:
no_done_list.sort()
len(no_done_list)

0

In [12]:
no_done_list

[]